In [445]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro, mannwhitneyu, ttest_ind

К вам попали результаты A/A/B-тестирования от одного известного маркетплейса. 

sample_a, sample_c — АА-группы, sample_b — отдельная группа. 

В каждом датасете есть три типа действий пользователей: 0 — клик, 1 — просмотр и 2 — покупка (пользователь просматривает выдачу товаров, кликает на понравившийся товар и совершает покупку).

Маркетплейс ориентируется на следующие метрики:

* ctr (отношение кликов к просмотрам товаров);
* purchase rate (отношение покупок к просмотрам товаров);
* gmv (оборот, сумма произведений количества покупок на стоимость покупки), где считаем 1 сессию за 1 точку (1 сессия на 1 пользователя).

In [446]:
# загрузка данных
df_a = pd.read_csv("data/sample_a.csv")
df_b = pd.read_csv("data/sample_b.csv")
df_c = pd.read_csv("data/sample_c.csv")
df = pd.read_csv("data/item_prices.csv")

## Подготовка данных

Замена действия просмотра с цифры 1 на цифру 5 для удобной проверки, что нет ситуаций , когда происходит покупка/клик без действия просмотра.

In [447]:
df_a.loc[df_a.action_id==1, 'action_id'] = 5
df_b.loc[df_b.action_id==1, 'action_id'] = 5
df_c.loc[df_c.action_id==1, 'action_id'] = 5

In [448]:
print(df_a.head())
print(df_b.head())
print(df_c.head())
print(df.head())

   user_id  item_id  action_id
0    84636      360          5
1    21217     9635          5
2    13445     8590          5
3    38450     5585          5
4    14160     2383          0
   user_id  item_id  action_id
0   118375     4105          5
1   107569     8204          5
2   175990      880          5
3   160582     9568          0
4   123400     4000          5
   user_id  item_id  action_id
0   274623     2863          5
1   265472      343          5
2   242779     6009          0
3   275009     2184          5
4   268104     3134          2
   item_id  item_price
0      338        1501
1       74         647
2     7696         825
3      866         875
4     5876         804


In [449]:
print(df_a.shape)
print(df_b.shape)
print(df_c.shape)
print(df.shape)

(1188912, 3)
(1198438, 3)
(1205510, 3)
(1000, 2)


In [450]:
df_a_group = df_a.groupby(['user_id', 'item_id'])['action_id'].sum()
df_b_group = df_b.groupby(['user_id', 'item_id'])['action_id'].sum()
df_c_group = df_c.groupby(['user_id', 'item_id'])['action_id'].sum()

Действие просмотра теперь равно пяти, поэтому любая сумма меньше 5-ти будет без действия просмотра, что является недопустимым.

In [451]:
print(df_a_group[df_a_group < 5])
print(df_b_group[df_b_group < 5])
print(df_c_group[df_c_group < 5])

Series([], Name: action_id, dtype: int64)
Series([], Name: action_id, dtype: int64)
Series([], Name: action_id, dtype: int64)


Ситуаций, когда происходит покупка/клик без действия просмотра, не обнаружено.

In [452]:
# полные дубликаты
print(df_a.duplicated().sum())
print(df_b.duplicated().sum())
print(df_c.duplicated().sum())
print(df.duplicated().sum())

0
0
0
0


Поиск дублей между группами

In [453]:
# item_id
list_a = list(set(df_a.item_id.values))
list_b = list(set(df_b.item_id.values))
list_c = list(set(df_c.item_id.values))
list_ab = list_a+list_b
list_ac = list_a+list_c
list_cb = list_c+list_b
print(f'Дубли номеров сессий(группы a, b): {not(len(set(list_ab))==len(list_ab))}')
print(f'Дубли номеров сессий(группы a, c): {not(len(set(list_ac))==len(list_ac))}')
print(f'Дубли номеров сессий(группы c, b): {not(len(set(list_cb))==len(list_cb))}')
print(f'Количество номеров сессий в группах a, b, с: {len(list_a), len(list_b), len(list_c)}')
print(f'Дубли номеров сессий(группы a, b): {len(list_ab)-len(set(list_ab))}')
print(f'Дубли номеров сессий(группы a, c): {len(list_ac)-len(set(list_ac))}')
print(f'Дубли номеров сессий(группы c, b): {len(list_cb)-len(set(list_cb))}')

Дубли номеров сессий(группы a, b): True
Дубли номеров сессий(группы a, c): True
Дубли номеров сессий(группы c, b): True
Количество номеров сессий в группах a, b, с: (955, 955, 955)
Дубли номеров сессий(группы a, b): 955
Дубли номеров сессий(группы a, c): 955
Дубли номеров сессий(группы c, b): 955


Номера сессий полностью совпадают, очевидно что это нормальная ситуация.

In [454]:
# user_id
list_a = list(set(df_a.user_id.values))
list_b = list(set(df_b.user_id.values))
list_c = list(set(df_c.user_id.values))
list_ab = list_a+list_b
list_ac = list_a+list_c
list_cb = list_c+list_b
print(f'Дубли уникальных пользователей(группы a, b): {not(len(set(list_ab))==len(list_ab))}')
print(f'Дубли уникальных пользователей(группы a, c): {not(len(set(list_ac))==len(list_ac))}')
print(f'Дубли уникальных пользователей(группы c, b): {not(len(set(list_cb))==len(list_cb))}')
print(f'Количество уникальных пользователей в группах a, b, с: {len(list_a), len(list_b), len(list_c)}')

Дубли уникальных пользователей(группы a, b): False
Дубли уникальных пользователей(группы a, c): False
Дубли уникальных пользователей(группы c, b): False
Количество уникальных пользователей в группах a, b, с: (996, 996, 994)


Дубликатов уникальных пользователей по разным группам нет - пользователи в каждой группе уникальны.

In [455]:
len(set(df.item_id))-len(df.item_id)

-45

45 товаров с задвоенной ценой...

In [456]:
# кодирование столбца с действиями
df_aa = pd.get_dummies(df_a.action_id)
df_bb = pd.get_dummies(df_b.action_id)
df_cc = pd.get_dummies(df_c.action_id)
df_aa = df_aa.rename(columns={0:'click', 2:'buy', 5:'view'})
df_bb = df_bb.rename(columns={0:'click', 2:'buy', 5:'view'})
df_cc = df_cc.rename(columns={0:'click', 2:'buy', 5:'view'})
df_a = pd.concat([df_a, df_aa], axis=1)
df_b = pd.concat([df_b, df_bb], axis=1)
df_c = pd.concat([df_c, df_cc], axis=1)

In [457]:
# добавление столбца с ценой при покупке
df_dict = df.set_index('item_id')
df_a['item_price'] = 0
df_a.loc[df_a.buy==True, 'item_price'] = df_a.loc[df_a.buy==True, 'item_id']
df_a.item_price = df_a.item_price.apply(lambda x: df_dict.loc[x, 'item_price'].mean() if x!=0 else 0)

df_b['item_price'] = 0
df_b.loc[df_b.buy==True, 'item_price'] = df_b.loc[df_b.buy==True, 'item_id']
df_b.item_price = df_b.item_price.apply(lambda x: df_dict.loc[x, 'item_price'].mean() if x!=0 else 0)

df_c['item_price'] = 0
df_c.loc[df_c.buy==True, 'item_price'] = df_c.loc[df_c.buy==True, 'item_id']
df_c.item_price = df_c.item_price.apply(lambda x: df_dict.loc[x, 'item_price'].mean() if x!=0 else 0)

## Расчет метрик

### ctr

In [458]:
df_actr = df_a.groupby('user_id')['click'].sum()/df_a.groupby('user_id')['view'].sum()*100
df_bctr = df_b.groupby('user_id')['click'].sum()/df_b.groupby('user_id')['view'].sum()*100
df_cctr = df_c.groupby('user_id')['click'].sum()/df_c.groupby('user_id')['view'].sum()*100

In [459]:
df_actr.describe()

count    996.000000
mean      20.000012
std        1.301818
min       16.544503
25%       19.057592
50%       19.895288
75%       20.837696
max       24.188482
dtype: float64

In [460]:
df_cctr.describe()

count    994.000000
mean      20.999952
std        1.255021
min       17.277487
25%       20.125786
50%       21.047120
75%       21.780105
max       25.235602
dtype: float64

In [461]:
df_bctr.describe()

count    996.000000
mean      16.000046
std        1.220117
min       11.727749
25%       15.183246
50%       16.020942
75%       16.858639
max       19.581152
dtype: float64

В группе В среднее значение метрики существенно ниже, в группах А и С значения почти одинаковы. Это значит что в группе В меньшее количество пользователей кликнуло на товар при просмотре.

### purchase rate

In [462]:
df_apurchase = df_a.groupby('user_id')['buy'].sum()/df_a.groupby('user_id')['view'].sum()*100
df_bpurchase = df_b.groupby('user_id')['buy'].sum()/df_b.groupby('user_id')['view'].sum()*100
df_cpurchase = df_c.groupby('user_id')['buy'].sum()/df_c.groupby('user_id')['view'].sum()*100

In [463]:
df_apurchase.describe()

count    996.000000
mean       4.999953
std        0.713566
min        2.827225
25%        4.502618
50%        4.921466
75%        5.445026
max        7.225131
dtype: float64

In [464]:
df_cpurchase.describe()

count    994.000000
mean       5.999975
std        0.772815
min        3.664921
25%        5.445026
50%        5.968586
75%        6.492147
max        9.214660
dtype: float64

In [465]:
df_bpurchase.describe()

count    996.000000
mean       9.999986
std        0.979052
min        6.910995
25%        9.319372
50%       10.052356
75%       10.680628
max       13.298429
dtype: float64

В группе В среднее значение метрики выше на 4 пункта, в группах А и С значения почти одинаковы. Значит что из всех просматривающих выдачу товаров пользователей в группе В большее количество пользователей совершило покупку, а если учесть предыдущую метрику(самое маленькое количество кликов к просмотру) то становится понятно что из кликнувших на товар пользователей достаточно большое количество купивших.

### gmv

In [466]:
df_agmv = df_a.groupby(['user_id', 'item_id'])['item_price'].sum()
df_bgmv = df_b.groupby(['user_id', 'item_id'])['item_price'].sum()
df_cgmv = df_c.groupby(['user_id', 'item_id'])['item_price'].sum()

In [467]:
df_agmv[df_agmv!=0].describe()

count    47556.000000
mean      1069.730237
std        553.449624
min        102.000000
25%        611.000000
50%       1054.000000
75%       1569.000000
max       1998.000000
Name: item_price, dtype: float64

In [468]:
df_cgmv[df_cgmv!=0].describe()

count    56953.000000
mean      1068.845375
std        553.570614
min        102.000000
25%        610.500000
50%       1054.000000
75%       1569.000000
max       1998.000000
Name: item_price, dtype: float64

In [469]:
df_bgmv[df_bgmv!=0].describe()

count    95114.000000
mean      1068.908650
std        556.063523
min        102.000000
25%        607.000000
50%       1054.000000
75%       1569.000000
max       1998.000000
Name: item_price, dtype: float64

Думаю уместнее смотреть средние на непустых данных. Количество покупок в группе В существенно больше количества покупок в группах А и С.

## Статистические тесты

In [470]:
alpha = 0.01

# функция для красивого вывода решения
def decision_hypothesis(p):
    print('p-value = {:.3f}'.format(p))
    if (p <= alpha):
        print("Отвергаем нулевую гипотезу в пользу альтернативной")
    else:
        print("У нас нет оснований отвергнуть нулевую гипотезу")
        
# функция для принятия решения о нормальности
def decision_normality(data):
    _, p = shapiro(data)
    print('p-value = {:.3f}'.format(p))
    if p <= alpha:
        print('p-значение меньше, чем заданный уровень значимости {:.2f}. Распределение отлично от нормального'.format(alpha))
    else:
        print('p-значение больше, чем заданный уровень значимости {:.2f}. Распределение является нормальным'.format(alpha))

### A/C

In [471]:
decision_normality(df_actr)
decision_normality(df_cctr)

p-value = 0.025
p-значение больше, чем заданный уровень значимости 0.01. Распределение является нормальным
p-value = 0.172
p-значение больше, чем заданный уровень значимости 0.01. Распределение является нормальным


In [472]:
decision_normality(df_apurchase)
decision_normality(df_cpurchase)

p-value = 0.006
p-значение меньше, чем заданный уровень значимости 0.01. Распределение отлично от нормального
p-value = 0.023
p-значение больше, чем заданный уровень значимости 0.01. Распределение является нормальным


In [473]:
decision_normality(df_agmv)
decision_normality(df_cgmv)

p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.01. Распределение отлично от нормального
p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.01. Распределение отлично от нормального


c:\Users\anzel\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 951130.
  res = hypotest_fun_out(*samples, **kwds)
c:\Users\anzel\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 949221.
  res = hypotest_fun_out(*samples, **kwds)


*Нулевая гипотеза* ($H_0$): метрика группы A равна метрике группы С.

$$ H_0 : p_a = p_с$$

*Альтернативная гипотеза* ($H_1$): метрика группы A отличается от метрики группы С.

$$ H_1 : p_a \neq p_c$$

In [474]:
alpha = 0.05

results = ttest_ind(
    df_actr.values,
    df_cctr.values,
    alternative='two-sided'
)
decision_hypothesis(results.pvalue)

p-value = 0.000
Отвергаем нулевую гипотезу в пользу альтернативной


In [475]:
results = mannwhitneyu(
    df_apurchase.values,
    df_cpurchase.values,
    alternative='two-sided'
)
decision_hypothesis(results.pvalue)

p-value = 0.000
Отвергаем нулевую гипотезу в пользу альтернативной


In [476]:
results = mannwhitneyu(
    df_agmv,
    df_cgmv,
    alternative='two-sided'
)
decision_hypothesis(results.pvalue)

p-value = 0.000
Отвергаем нулевую гипотезу в пользу альтернативной


По результатам тестирования есть статистически значимая разница между группами A и C по всем метрикам. Получается что выборки неравномерны и сплиты "разъезжаются".

### A/B

*Нулевая гипотеза* ($H_0$): метрика группы A равна метрике группы В.

$$ H_0 : p_a = p_b$$

*Альтернативная гипотеза* ($H_1$): метрика группы A отличается от метрики группы B.

$$ H_1 : p_a \neq p_b$$

In [477]:
alpha = 0.01

decision_normality(df_actr)
decision_normality(df_bctr)

p-value = 0.025
p-значение больше, чем заданный уровень значимости 0.01. Распределение является нормальным
p-value = 0.332
p-значение больше, чем заданный уровень значимости 0.01. Распределение является нормальным


In [478]:
decision_normality(df_apurchase)
decision_normality(df_cpurchase)

p-value = 0.006
p-значение меньше, чем заданный уровень значимости 0.01. Распределение отлично от нормального
p-value = 0.023
p-значение больше, чем заданный уровень значимости 0.01. Распределение является нормальным


In [479]:
decision_normality(df_agmv)
decision_normality(df_cgmv)

p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.01. Распределение отлично от нормального
p-value = 0.000
p-значение меньше, чем заданный уровень значимости 0.01. Распределение отлично от нормального


In [480]:
alpha = 0.05
results = ttest_ind(
    df_actr.values,
    df_bctr.values,
    alternative='two-sided'
)
decision_hypothesis(results.pvalue)

p-value = 0.000
Отвергаем нулевую гипотезу в пользу альтернативной


In [481]:
results = mannwhitneyu(
    df_apurchase.values,
    df_bpurchase.values,
    alternative='two-sided'
)
decision_hypothesis(results.pvalue)

p-value = 0.000
Отвергаем нулевую гипотезу в пользу альтернативной


In [482]:
results = mannwhitneyu(
    df_agmv,
    df_bgmv,
    alternative='two-sided'
)
decision_hypothesis(results.pvalue)

p-value = 0.000
Отвергаем нулевую гипотезу в пользу альтернативной


## Вывод

Метрики в группе В отличаются от метрик в группе А. Здесь можно было бы посмотреть в какую сторону отличаются, но смысла это не имеет потому что контрольные группы A и C имеют статистически значимую разницу, а значит доверять результатам тестирования нельзя.